# ImageFolder

In [18]:
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader

In [19]:
from matplotlib.pyplot import imshow
%matplotlib inline

In [20]:
trans = transforms.Compose([ # transform을 여러번 할 거면 이렇게 묶어서 할 수 있다.
    transforms.Resize((64,128))
])

# 제목에 _ 뒤에 있는 숫자가 label. 
# root 이하의 폴더를 recursive하게 조사한다.
train_data = torchvision.datasets.ImageFolder(root='./custom_data/origin_data',
                                              transform=trans)

In [22]:
for num, value in enumerate(train_data):
    data, label = value
    print(num, data, label)
    
    if(label == 0):
        data.save('custom_data/train_data/gray/%d_%d.jpeg'%(num, label))
    else:
        data.save('custom_data/train_data/red/%d_%d.jpeg'%(num, label))

# Usage of ImageFolder

In [39]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms

In [40]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [41]:
trans = transforms.Compose([
    transforms.ToTensor() # PIL to Tensor
])

train_data = torchvision.datasets.ImageFolder(root='./custom_data/train_data',
                                              transform=trans)

data_loader = DataLoader(dataset=train_data, # ImageFolder를 이용해서 가져왔다.
                         batch_size=32,
                         shuffle=True,
                         drop_last=True,
                         num_workers=2) # 멀티프로세싱 개수

In [42]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.FC = nn.Sequential( # need flatten before input
            nn.Linear(16*13*29, 120),
            nn.ReLU(),
            nn.Linear(120,2)
        )
        # sigmoid after
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.shape[0], -1)
        out = self.FC(out)
        return out

#testing 
net = CNN().to(device)
test_input = (torch.Tensor(1,3,64,128)).to(device)
test_out = model(test_input)
test_out

tensor([[nan, nan]], grad_fn=<AddmmBackward0>)

In [43]:
optimizer = optim.Adam(net.parameters(), lr = 0.001)
loss_func = nn.CrossEntropyLoss().to(device)

In [44]:
total_batch = len(data_loader)

training_epochs = 20
for epoch in range(1, training_epochs + 1):
    avg_cost = 0
    for num, data in enumerate(data_loader):
        imgs, label = data
        imgs = imgs.to(device)
        label = label.to(device)
        
        optimizer.zero_grad()
        hypothesis = net(imgs)
        loss = loss_func(hypothesis, label)
        loss.backward()
        optimizer.step()
        avg_cost += loss / total_batch
    print('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))

[Epoch:2] cost = 0.5089213252067566
[Epoch:3] cost = 0.05813751369714737
[Epoch:4] cost = 0.0027425051666796207
[Epoch:5] cost = 0.0002715228183660656
[Epoch:6] cost = 0.00014487621956504881
[Epoch:7] cost = 9.05981651158072e-05
[Epoch:8] cost = 7.206144073279575e-05
[Epoch:9] cost = 6.118701276136562e-05
[Epoch:10] cost = 5.15760802954901e-05
[Epoch:11] cost = 5.329633131623268e-05
[Epoch:12] cost = 3.35050790454261e-05
[Epoch:13] cost = 2.810091791616287e-05
[Epoch:14] cost = 2.005798160098493e-05
[Epoch:15] cost = 1.5580319086438976e-05
[Epoch:16] cost = 1.2331304787949193e-05
[Epoch:17] cost = 9.922556273522787e-06
[Epoch:18] cost = 8.26423547550803e-06
[Epoch:19] cost = 7.0877536018087994e-06
[Epoch:20] cost = 6.202488748385804e-06
[Epoch:21] cost = 5.4202027968131006e-06


In [47]:
# model 저장 및 load

torch.save(model.state_dict(), "./model/model.pth")
new_model = CNN().to(device) # 모델의 구성은 동일해야한다.
new_model.load_state_dict(torch.load('./model/model.pth'))

<All keys matched successfully>

In [48]:
# 저장된 것 비교
print(net.layer1[0], new_model.layer1[0])
print(net.layer1[0].weight[0,0,0], new_model.layer1[0].weight[0,0,0])

net.layer1[0].weight[0] == new_model.layer1[0].weight[0]

Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1)) Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
tensor([-0.0929,  0.0014, -0.0193, -0.0239,  0.0900],
       grad_fn=<SelectBackward0>) tensor([ 0.0595, -0.0510, -0.0224,  0.0542, -0.1087],
       grad_fn=<SelectBackward0>)


tensor([[[False, False, False, False, False],
         [False, False, False, False, False],
         [False, False, False, False, False],
         [False, False, False, False, False],
         [False, False, False, False, False]],

        [[False, False, False, False, False],
         [False, False, False, False, False],
         [False, False, False, False, False],
         [False, False, False, False, False],
         [False, False, False, False, False]],

        [[False, False, False, False, False],
         [False, False, False, False, False],
         [False, False, False, False, False],
         [False, False, False, False, False],
         [False, False, False, False, False]]])

In [51]:
trans = transforms.Compose([
    transforms.Resize((64,128)),
    transforms.ToTensor()
])
test_data = torchvision.datasets.ImageFolder(
    root='./custom_data/test_data',
    transform=trans
    )
test_set = DataLoader(dataset=test_data, batch_size=len(test_data))

In [53]:
with torch.no_grad():
    for num, data in enumerate(test_set):
        imgs, label = data
        imgs = imgs.to(device)
        label = label.to(device)
        
        prediction = net(imgs)
        accuracy = (torch.argmax(prediction, 1) == label).float().mean()
    print(accuracy.item())

1.0
